In [1]:
from lxml import html
import requests
from string import ascii_lowercase
import sys
import json
from xml.etree.ElementTree import tostring
from IPython.display import clear_output
from time import sleep

# base URL
page_root_url = 'http://www.jokeswarehouse.com/'
# file name for the topics
topics_filename = 'topicsJW.json'
# filename for the jokes
jokes_filename = 'jokesJW.json'
# max joke length
MAX_JOKE_LENGTH = 1000

In [2]:
# request HTML page
page_response = requests.get(page_root_url)
# convert to tree for searching for elements with xpath
tree = html.fromstring(page_response.content)

topics_names = tree.xpath('//td[@bgcolor="#FFFFFF"]//td[@valign="top"]//ul//li//text()')
topics_urls = tree.xpath('//td[@bgcolor="#FFFFFF"]//td[@valign="top"]//ul//li//a/@href')

topics = {}
ix = 0
for topic_name in topics_names:
    if len(topic_name.strip()) > 1:
        topics[topic_name] = topics_urls[ix]
        ix += 1
        
with open(topics_filename, 'w') as fp:
    json.dump(topics, fp)

In [3]:
with open(topics_filename, 'r') as fp:
    topics = json.load(fp)  
topic_names = sorted(topics.keys())
joke_topic_names = [topic_names[0]]

jokes = {}
tot_n_jokes = 0
try:    
    with open(jokes_filename, 'r') as fp:
        jokes = json.load(fp)    
    if len(jokes) == 0:
        print('File does not contain jokes of jokes dict is empty')
    else:
        topics_without_jokes = []
        for topic in topics:
            if topic in jokes.keys():
                n_jokes_topic = 0
                jokes_topic = jokes[topic]
                for joke in jokes_topic:
                    n_jokes_topic+=1
            else:
                topics_without_jokes.append(topic)
            tot_n_jokes += n_jokes_topic
except:
    print('Could not find jokes file: ' + jokes_filename)
    
print('Total number of topics: ' + str(len(topic_names)))
print('Total number of jokes: ' + str(tot_n_jokes))
curr_topic = joke_topic_names[-1]
print('Current topic: ' + curr_topic)
print('Last topic: ' + topic_names[-1])

Total number of topics: 28
Total number of jokes: 5964
Current topic: Animal Jokes
Last topic: Workplace Jokes


In [4]:
jokes = {}

for topic_key in topics.keys():
    page_response = requests.get(topics[topic_key], stream=True)
    tree = html.fromstring(page_response.content)
    joke_links = tree.xpath('//td[@bgcolor="#FFFFFF"]//center//a/@href')
    jokes_list = []
    ix = 0
    for joke_link in joke_links:
        ix += 1
        page_response = requests.get(joke_link, stream=True)
        tree = html.fromstring(page_response.content)
        joke_html = tree.xpath('//td[@bgcolor="#FFFFFF"]//p//text()')
        joke_text = ''
        for joke_html_in in joke_html:
            joke_text += joke_html_in.strip()
        formated_joke = joke_text.strip()
        print('Current topic: ' + str(topic_key))
        print('Total number of jokes to process: ' + str(ix) + '/' + str(len(joke_links)))
        print(formated_joke)
        clear_output(wait=True)
        jokes_list.append(formated_joke)
    jokes[topic_key] = jokes_list
    with open(jokes_filename, 'w') as fp:
        json.dump(jokes, fp)
    sleep(3)

Current topic: Workplace Jokes
Total number of jokes to process: 290/290
Dear Employees:It has been brought to management's attention that some individuals throughout the Service have been using foul language during the course of normal conversation with their co-workers.Due to complaints received from some employees who may be easily offended, this type of language will no longer be tolerated.We do, however, realize the critical importance of being able to accurately express your feelings when communicating with co-workers.Therefore,a list of 18 New and Innovative 'TRY SAYING' phrases have been provided so that proper exchange of ideas and information can continue in an effective manner.Number 1TRY SAYING: I think you could use more training..INSTEAD OF: You don't know what the f___ you're doing.Number 2TRY SAYING: She's an aggressive go-getter.INSTEAD OF: She's a f___ing bit__.Number 3TRY SAYING: Perhaps I can work late.INSTEAD OF: And when the f___ do you expect me to do this?Number